# Analytic Hierarchy Process (AHP): House Hunting Example

This notebook walks you through each step of the Analytic Hierarchy Process (AHP) method for choosing a house, using the same data and values as in the [blog post](https://medium.com/@marciaolivetree/a-step-by-step-guide-to-ahp). 

## Table of Contents
1. [Setup](#setup)
2. [Step 1: Define Criteria and Options](#criteria-options)
3. [Step 2: Pairwise Comparison Matrix for Criteria](#pairwise-criteria)
4. [Step 3: Consistency Ratio Check](#consistency-ratio)
5. [Step 4: Compute Criteria Weights](#criteria-weights)
4. [Step 5: Pairwise Comparisons for Options (per Criterion)](#pairwise-options)
6. [Step 6: Compute Local Priority Scores](#local-priority)
7. [Step 7: Compute Global Scores and Rank Options](#global-priority)

## Setup 

We'll use the [`ahpy`](https://github.com/PhilipGriffith/AHPy) Python library for calculations, and `numpy` for handling matrices.

```python
# Install ahpy and pandas, if you haven't done it already
# !pip install ahpy
# !pip install pandas

In [1]:
import ahpy
import numpy as np
import pandas as pd

## <a name="criteria-options"></a>Step 1: Define Criteria and Options

**Decision Criteria:**
- Neighbourhood Safety
- Location (distance from work, in km)
- House Condition
- Price (€)
- House Size ($m^2$)

**Options/Alternatives (aka the decision set):**
- House A: Modern Apartment Downtown
- House B: Suburban Family House
- House C: Rural Cottage

In [2]:
# Criteria
criteria = ['Safety', 'Location', 'Condition', 'Price', 'Size']

# Options
options = ['House A', 'House B', 'House C']

## <a name="pairwise-criteria"></a>Step 2: Pairwise Comparison Matrix for Criteria

Here is the pairwise comparison matrix for our five criteria:


| Criteria      | Safety | Location | Condition | Price | Size |
| ------------- | :----: | :------: | :-------: | :---: | :--: |
| **Safety**    |    1   |     3    |     5     |   7   |   9  |
| **Location**  |   1/3  |     1    |     2     |   5   |   7  |
| **Condition** |   1/5  |    1/2   |     1     |   2   |   4  |
| **Price**     |   1/7  |    1/5   |    1/2    |   1   |   3  |
| **Size**      |   1/9  |    1/7   |    1/4    |  1/3  |   1  |

Let's define the pairwise comparisons dictionary for ahpy:

In [3]:
criteria_comparisons = {
    ('Safety', 'Location'): 3,
    ('Safety', 'Condition'): 5,
    ('Safety', 'Price'): 7,
    ('Safety', 'Size'): 9,
    ('Location', 'Condition'): 2,
    ('Location', 'Price'): 5,
    ('Location', 'Size'): 7,
    ('Condition', 'Price'): 2,
    ('Condition', 'Size'): 4,
    ('Price', 'Size'): 3,
}

criteria_matrix = ahpy.Compare('Criteria', criteria_comparisons, precision=3)

## <a name="consistency-ratio"></a>Step 3: Consistency Ratio Check

Before computing the criteria weights, let's first check the consistency of the pairwise comparisons. This ensures the judgments are logical and reliable.

We will use AHP's built-in consistency check based on the Consistency Ratio (CR). This metric compares the decision maker's judgments against a large set of randomly generated ones to assess how logically coherent the inputs are.

**Consistency Ratio (CR):**
$$
CR = \frac{CI}{RI}
$$

**Consistency Index (CI):**
$$
CI = \frac{\lambda_{max} - n}{n - 1}
$$

Where:
- $\lambda_{max}$: the maximum eigenvalue of the pairwise comparison matrix  
- $n$: number of criteria  
- $RI$: Random Consistency Index (depends on the size of the matrix; see Saaty's RI table)


### How to interpret the Consistency Ratio
- **CR ≤ 0.10:** Judgments are acceptably consistent. Proceed.
- **CR > 0.10:** Review comparisons for contradictions.

In [4]:
print('\nConsistency Ratio:', criteria_matrix.consistency_ratio)


Consistency Ratio: 0.035


For our house hunting example, the consistency ratio is 0.035, which is well below the recommended 0.10 threshold. This indicates that the decision maker's judgments are acceptably consistent.

## <a name="criteria-weights"></a>Step 4: Compute Criteria Weights

Now, let's extract the weights for each criterion (rounded to 3 decimals) using AHP's eigenvector method:

In [5]:
criteria_weights = criteria_matrix.target_weights

# Convert to dataframe
weights_df = pd.DataFrame.from_dict(criteria_weights, orient='index', columns=["Criteria Weights"])

# Display as a table
display(weights_df)

,Criteria Weights
Safety,0.524
Location,0.247
Condition,0.123
Price,0.071
Size,0.036


The most important criteria for the decision maker is Neighbourhood Safety, followed by House Location (proximity to work).

## <a name="pairwise-options"></a>Step 5: Pairwise Comparisons for Options (per Criterion)

At this stage, we evaluate how well each house performs under each criterion.
- For **qualitative criteria**, like _Safety_ and _Condition_, we use **pairwise comparison matrices**. This approach allows us to capture nuanced, subjective judgments.
- For **objective criteria** such as _Price_, _Location_, and _Size_, we use the actual data (direct ratings where lower price, closer location, or larger size are directly reflected in the scoring).


#### Pairwise Comparisons – Condition:

|            | House A | House B | House C |
|------------|---------|---------|---------|
| **House A**|   1     |   5     |   9     |
| **House B**|  1/5    |   1     |   4     |
| **House C**|  1/9    |  1/4    |   1     |

#### Pairwise Comparisons – Safety:

|            | House A | House B | House C |
|------------|---------|---------|---------|
| **House A**|   1     |   5     |   1     |
| **House B**|  1/5    |   1     |  1/5    |
| **House C**|   1     |   5     |   1     |

In [6]:
condition_comparisons = {
    ('House A', 'House B'): 5,
    ('House A', 'House C'): 9,
    ('House B', 'House C'): 4,
}
condition_matrix = ahpy.Compare('Condition', condition_comparisons, precision=3)
sorted_condition_priorities = dict(sorted(condition_matrix.local_weights.items()))
print('Condition local priorities   :', sorted_condition_priorities)

Condition local priorities   : {'House A': 0.743, 'House B': 0.194, 'House C': 0.063}


In [7]:
safety_comparison = {
    ('House A', 'House B'): 5,
    ('House C', 'House B'): 5,  # House C vs. B
    ('House A', 'House C'): 1   # A equals C
}
safety_matrix = ahpy.Compare('Safety', safety_comparison, precision=3)
sorted_safety_priorities = dict(sorted(safety_matrix.local_weights.items()))
print('Safety local priorities   :', sorted_safety_priorities)

Safety local priorities   : {'House A': 0.455, 'House B': 0.091, 'House C': 0.455}


## <a name="local-priority"></a>Step 6: Compute Local Priority Scores

Now, let's assemble all local priority scores for each house under each criterion. For objective criteria (Price, Location, Size) we'll use normalized scores (lowest price/best location = 1).


| Criteria/Options                | House A (Modern Apartment Downtown) | House B (Suburban Family House) | House C (Rural Cottage)    |
|---------------------------------|-------------------------------------|----------------------------------|----------------------------|
| **Price (€)**                   | 420k                                | 380k                             | 320k                       |
| **Location (distance to work, km)** | 5km                              | 21km                             | 47km                       |
| **House Size (m²)**             | 75m²                                | 190m²                            | 285m²                      |
| **Condition**                   | New                                 | Good Shape                       | Needs renovation           |
| **Neighbourhood Safety**        | Safe                                | Moderate                         | Safe                       |

In [8]:
# Input values:
price = np.array([420e3, 380e3, 320e3])
location = np.array([5, 21, 47])
size = np.array([75, 190, 285])

# Normalize: invert for 'lower is better'
price_norm = 1 / price
location_norm = 1 / location
size_norm = size  # 'higher is better'

# Scale each series so that sum = 1
def normalize(vec):
    return vec / vec.sum()

price_lp = normalize(price_norm)
location_lp = normalize(location_norm)
size_lp = normalize(size_norm)

local_table = pd.DataFrame({
    'Price': price_lp,
    'Location': location_lp,
    'Size': size_lp
}, index=['House A', 'House B', 'House C'])

display(local_table.round(3))

,Price,Location,Size
House A,0.293,0.744,0.136
House B,0.323,0.177,0.345
House C,0.384,0.079,0.518


Now merge with the Condition and Safety local priorities like this:

In [9]:
local_priorities = local_table.copy()
local_priorities['Condition'] = sorted_condition_priorities.values()
local_priorities['Safety'] = sorted_safety_priorities.values()
local_priorities.round(3)

,Price,Location,Size,Condition,Safety
House A,0.293,0.744,0.136,0.743,0.455
House B,0.323,0.177,0.345,0.194,0.091
House C,0.384,0.079,0.518,0.063,0.455


## <a name="global-priority"></a>Step 7: Compute Global Scores and Rank Options

In this last step, we combine the criteria weights with the local priority scores for each house to get the global priority scores and decide on the best compromise solution.


The formula we'll be using is:

$$
\text{Global Priority}_j = \sum_{i=1}^{n} (w_i \times p_{ij})
$$

Where:
- $w_i$: weight of criterion \( i \)
- $p_{ij}$: local priority score of option \( j \) under criterion \( i \)
- $n$: umber of criteria

This calculation gives each house a single score that reflects both the importance of each criterion and how well the house performs on each one. The option with the highest global priority is your best overall choice.

In [10]:
global_scores = {house: 0 for house in options}

for criterion, weights in criteria_weights.items():
    for house in options:
        global_scores[house] += weights * local_priorities[criterion][house]

# Normalize global scores to sum to 1
total = sum(global_scores.values())
for house in global_scores:
    global_scores[house] /= total

print('Global Priority Scores:')
for house, score in sorted(global_scores.items(), key=lambda x: x[1], reverse=True):
    print(f'{house}: {score:.2f}')

Global Priority Scores:
House A: 0.54
House C: 0.31
House B: 0.15


## Options Ranked!

| House   | Global Priority Score |
|---------|----------------------|
| House A | 0.54                 |
| House C | 0.31                 |
| House B | 0.15                 |

**Conclusion:**  
And the winner is… House A! The modern apartment downtown takes the crown as the best compromise in this decision.